In [2]:
# ─── 1. IMPORTS & LOAD INTERMEDIATE DF ──────────────────────────────────────────
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("../data/processed/cleaned_loans.csv")  # or load df4 if you persisted it
# If df4 isn’t on disk yet, load your previous notebook’s output instead:
# df = pd.read_pickle("../data/processed/intermediate_df.pkl")


In [3]:
# ─── 3. ORDINAL ENCODING ────────────────────────────────────────────────────────
# Grade (A–G → 1–7)
grade_order = sorted(df['grade'].unique())
grade_map   = {g: i+1 for i, g in enumerate(grade_order)}
df['grade_enc'] = df['grade'].map(grade_map)

# Sub-grade (A1…G5 → 1–35)
sub_order   = sorted(df['sub_grade'].unique(), key=lambda x: (x[0], int(x[1:])))
sub_map     = {sg: i+1 for i, sg in enumerate(sub_order)}
df['sub_grade_enc'] = df['sub_grade'].map(sub_map)

# Drop originals
df.drop(columns=['grade', 'sub_grade'], inplace=True)


In [4]:
# ─── 4. ONE-HOT ENCODE NOMINALS ────────────────────────────────────────────────
nominals = [
    'home_ownership', 'verification_status', 'pymnt_plan', 'purpose',
    'addr_state', 'initial_list_status', 'application_type',
    'hardship_flag', 'disbursement_method', 'debt_settlement_flag'
]
df = pd.get_dummies(df, columns=nominals, drop_first=True)


In [5]:
# ─── 5. SCALING NUMERICAL FEATURES ──────────────────────────────────────────────
# Identify numeric cols minus the target
num_cols = df.select_dtypes(include=['float64','int64']).columns.tolist()
num_cols.remove('default_flag')  # leave target unscaled

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])


In [6]:
# 1. Shape & columns
print("Final shape:", df.shape)
print("Columns:", df.columns.tolist())


Final shape: (2260701, 138)
Columns: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'open_acc_6m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_il_high_credit_limit', 'default_flag', 'l

In [7]:
# 2. No missing values
nulls = df.isnull().sum()
print("Nulls per column (should all be 0):\n", nulls[nulls > 0])


Nulls per column (should all be 0):
 emp_length            146940
open_acc_6m           866163
max_bal_bc            866162
all_util              866381
inq_fi                866162
total_cu_tl           866163
inq_last_12m          866163
mo_sin_old_il_acct    139104
dtype: int64


In [8]:
# 3. Target distribution
print("Default flag distribution:\n", df['default_flag'].value_counts(normalize=True))


Default flag distribution:
 default_flag
0    0.869768
1    0.130232
Name: proportion, dtype: float64


In [9]:
# 4. Peek at head & dtypes
display(df.head())
print("\nData types:\n", df.dtypes.value_counts())


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,...,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_w,application_type_Joint App,hardship_flag_Y,disbursement_method_DirectPay,debt_settlement_flag_Y
0,-1.245558,-1.245233,-1.242719,-0.635884,0.185670,-1.208122,1.102185,-0.204021,-0.910816,-0.353856,...,False,False,False,False,False,True,False,False,False,False
1,1.050371,1.051154,1.052689,-0.635884,-0.228228,1.401624,1.102185,-0.115286,-0.194912,0.799254,...,False,False,False,False,False,True,False,False,False,False
2,0.538956,0.539636,0.541389,1.572615,-0.478637,-0.049204,1.102185,-0.133033,-0.567323,-0.353856,...,False,False,False,False,False,True,True,False,False,False
3,2.171133,2.172139,2.173196,1.572615,0.363647,1.437631,1.102185,0.284021,-0.124379,-0.353856,...,False,False,False,False,False,True,False,False,False,False
4,-0.505638,-0.505165,-0.502967,1.572615,1.936461,-0.583504,-0.815278,0.234622,0.461745,0.799254,...,False,False,False,False,False,True,False,False,False,False



Data types:
 bool       76
float64    61
int64       1
Name: count, dtype: int64


In [10]:
# 5. Quick stats on numeric features
display(df.describe().T)


,count,mean,std,min,25%,50%,75%,max
loan_amnt,2260701.0,3.163135e-17,1.0,-1.582874,-0.766786,-0.233608,0.538956,2.715192
funded_amnt,2260701.0,7.985784e-17,1.0,-1.582617,-0.766365,-0.235802,0.539636,2.716306
funded_amnt_inv,2260701.0,-1.531279e-16,1.0,-1.634353,-0.764056,-0.241878,0.541389,2.717131
term,2260701.0,8.999093e-17,1.0,-0.635884,-0.635884,-0.635884,1.572615,1.572615
int_rate,2260701.0,-1.839144e-15,1.0,-1.610649,-0.745601,-0.097850,0.599569,3.703808
...,...,...,...,...,...,...,...,...
loan_issue_year,2260701.0,-3.113898e-14,1.0,-4.941234,-0.478494,0.079348,0.637191,1.195033
loan_issue_month,2260701.0,7.643823e-18,1.0,-1.687174,-0.807289,0.072597,0.952483,1.539074
credit_history_years,2260701.0,-3.512890e-16,1.0,-2.069145,-0.669523,-0.203220,0.502295,8.705732
grade_enc,2260701.0,1.752548e-16,1.0,-1.322554,-0.527692,0.267170,0.267170,3.446619


In [11]:
# 6. Verify encoding of a few dummy columns
for col in df.columns:
    if col.startswith('home_ownership_') or col.startswith('purpose_'):
        print(col, df[col].unique())


home_ownership_MORTGAGE [ True False]
home_ownership_NONE [False  True]
home_ownership_OTHER [False  True]
home_ownership_OWN [False  True]
home_ownership_RENT [False  True]
purpose_credit_card [False  True]
purpose_debt_consolidation [ True False]
purpose_educational [False  True]
purpose_home_improvement [False  True]
purpose_house [False  True]
purpose_major_purchase [False  True]
purpose_medical [False  True]
purpose_moving [False  True]
purpose_other [False  True]
purpose_renewable_energy [False  True]
purpose_small_business [False  True]
purpose_vacation [False  True]
purpose_wedding [False  True]


In [17]:
# ─── Drop sparse columns & impute emp_length ────────────────────────────────────
to_drop_sparse = [
    'open_acc_6m', 'max_bal_bc', 'all_util',
    'inq_fi', 'total_cu_tl', 'inq_last_12m', 'mo_sin_old_il_acct'
]
df.drop(columns=to_drop_sparse, inplace=True)

# Impute emp_length with its median
median_emp = df['emp_length'].median()
df['emp_length'].fillna(median_emp, inplace=True)

# ─── Re-check for any nulls ─────────────────────────────────────────────────────
nulls_after = df.isnull().sum()
print("Columns still with nulls:\n", nulls_after[nulls_after > 0])


/var/folders/44/w7p6wqzx45xc0d33d14kpptw0000gn/T/ipykernel_2790/2247250766.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['emp_length'].fillna(median_emp, inplace=True)


Columns still with nulls:
 Series([], dtype: int64)


In [18]:
import os

# Ensure the processed folder exists
os.makedirs("../data/processed", exist_ok=True)

# Save to CSV
df.to_csv("../data/processed/cleaned_loans2.csv", index=False)

print("Cleaned dataset saved at data/processed/cleaned_loans.csv — shape:", df.shape)


Cleaned dataset saved at data/processed/cleaned_loans.csv — shape: (2260701, 131)
